In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import utils

from keras.models import Model, Input
from keras.layers import Dense, Conv1D, BatchNormalization, GlobalMaxPooling1D, Dropout, Embedding

from utils.preprocessing_utils import tokenize_sentences, convert_tokens_to_padded_sequence
from utils.dataset_utils import load_data_from_csv
from utils.embedding_utils import load_word2vec_embeddings, create_initial_embedding_matrix
from utils.training_utils import train_model

/home/philipp/miniconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import importlib
import utils
importlib.reload(utils.embedding_utils)
importlib.reload(utils.dataset_utils)
importlib.reload(utils.preprocessing_utils)
importlib.reload(utils.training_utils)
importlib.reload(utils.keras_utils)

<module 'utils.keras_utils' from '/home/philipp/work/gitprojects/toxic-comment-experiments/utils/keras_utils.py'>

Global parameters which hold for all models

In [3]:
random_seed = 2018
classes = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
features = 'comment_text'
np.random.seed(random_seed)
path_train_data = 'data/kaggle/train.csv'
path_test_data = 'data/kaggle/test_complete.csv'
path_tokenizer = 'data/models/word_tokenizer.pickle'

embedding_length = 300
path_embeddings = 'data/embeddings/GoogleNews-vectors-negative300.bin.gz'

Load train and test data and pretrained word2vec embeddings

In [4]:
X_train, Y_train = load_data_from_csv(path_train_data, features, classes)
X_test, Y_test = load_data_from_csv(path_test_data, features, classes)

emb_idx, emb_mean, emb_std = load_word2vec_embeddings(path_embeddings)

Preprocessing and tokenizatin of train and test data

In [5]:
X_train_tok = tokenize_sentences(X_train)
del X_train
X_test_tok = tokenize_sentences(X_test)
del X_test

Create initial embedding matrix for neural network and word -> idx mapping

In [6]:
embedding_matrix, word_embedding_mapping = create_initial_embedding_matrix(X_train_tok, X_test_tok, emb_idx, emb_mean, emb_std, embedding_length, debug=True)
del emb_idx

Number of unique tokens: 326175
Number of tokens found in pretrained embeddings: 74211


Transform comments in train and test data to padded matrices

In [7]:
max_len_train = X_train_tok.apply(lambda x: len(x)).max()
max_len_test = X_test_tok.apply(lambda x: len(x)).max()
# limit length to 2000, otherwise we get a MemoryError
max_comment_length = 2000 #max(max_len_train, 2000)
X_train_input = convert_tokens_to_padded_sequence(X_train_tok, word_embedding_mapping, max_comment_length)
del X_train_tok
X_test_input = convert_tokens_to_padded_sequence(X_test_tok, word_embedding_mapping, max_comment_length)
del X_test_tok

### Singlelayer CNN with a single window size

This simple CNN consists of an embedding layer, a single convolution layer with a fixed window size and a fully connected hidden layer.

In [8]:
m1_kernel_size = 3
m1_hidden_dim = 256
m1_num_filters = 150
m1_dropout = 0.5

The network architecture

In [12]:
m1_input = Input((max_comment_length,))
m1_word_emb = Embedding(input_dim=len(embedding_matrix), output_dim=embedding_length, input_length=max_comment_length, weights=[embedding_matrix])(m1_input)

m1_conv1 = Conv1D(kernel_size=m1_kernel_size, filters=m1_num_filters, padding='same')(m1_word_emb)
m1_conv1 = BatchNormalization()(m1_conv1)
m1_conv1 = GlobalMaxPooling1D()(m1_conv1)

m1_fc2 = Dense(m1_hidden_dim, activation='relu')(m1_conv1)
m1_dropout2 = Dropout(m1_dropout)(m1_fc2)
m1_output = Dense(len(classes), activation='sigmoid')(m1_dropout2)

m1_model = Model(inputs=[m1_input], outputs=[m1_output])
m1_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 2000)              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 2000, 300)         97852800  
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 2000, 150)         135150    
_________________________________________________________________
batch_normalization_2 (Batch (None, 2000, 150)         600       
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 150)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               38656     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
__________

Train model

In [13]:
m1_model, predictions = train_model(m1_model, X_train_input, Y_train, (X_test_input, Y_test), \
                                    4, 64, 'adam', 'binary_crossentropy', [], random_seed)

Train on 159571 samples, validate on 63978 samples
Epoch 1/4
159552/159571 [============================>.] - ETA: 0s - loss: 0.0708
 train: ROC-AUC - epoch: 0 - score: 0.989640

 val: ROC-AUC - epoch: 0 - score: 0.975268
159571/159571 [==============================] - 457s 3ms/step - loss: 0.0708 - val_loss: 0.0683
Epoch 2/4
159552/159571 [============================>.] - ETA: 0s - loss: 0.0415
 train: ROC-AUC - epoch: 1 - score: 0.994992

 val: ROC-AUC - epoch: 1 - score: 0.978218
159571/159571 [==============================] - 458s 3ms/step - loss: 0.0415 - val_loss: 0.0682
Epoch 3/4
159552/159571 [============================>.] - ETA: 0s - loss: 0.0340
 train: ROC-AUC - epoch: 2 - score: 0.996159

 val: ROC-AUC - epoch: 2 - score: 0.976215
159571/159571 [==============================] - 458s 3ms/step - loss: 0.0340 - val_loss: 0.0693
Epoch 4/4
159552/159571 [============================>.] - ETA: 0s - loss: 0.0286
 train: ROC-AUC - epoch: 3 - score: 0.997456

 val: ROC-AUC - e